In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spam").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/22 22:32:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/22 22:32:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## The data

In [2]:
data = spark.read.csv('smsspamcollection/SMSSpamCollection', inferSchema=True, sep='\t')

In [3]:
data.show(5)

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
+----+--------------------+
only showing top 5 rows



In [4]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [5]:
data.show(5)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
+-----+--------------------+
only showing top 5 rows



## Preparation and cleaning

In [6]:
from pyspark.sql.functions import length

In [7]:
data = data.withColumn('length', length(data['text']))

In [8]:
data.show(5)

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
+-----+--------------------+------+
only showing top 5 rows



In [9]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



We see that the length of a ham message is on average 71 charachters long. On the other hand, the average length of spam messages is almost twice that (~138 charachters). This means that the average text length might be a useful feature.

In [10]:
from pyspark.ml.feature import (Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer, VectorAssembler)

/Users/schanlar/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [11]:
# Split sentences into tokens/words
tokenizer = Tokenizer(inputCol='text', outputCol='tokenText')

# Remove all common words that are not very useful for our analysis
stop_remover = StopWordsRemover(inputCol='tokenText', outputCol='stopToken')

# Count vectorization
count_vec = CountVectorizer(inputCol='stopToken', outputCol='countVec')

# Inverse document frequency
idf = IDF(inputCol='countVec', outputCol='tfIDF')

ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

24/01/22 22:32:25 WARN StopWordsRemover: Default locale set was [en_GR]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


In [12]:
assembler = VectorAssembler(inputCols=['tfIDF', 'length'], outputCol='features')

## Create a Naive Bayes classification model

In [13]:
from pyspark.ml.classification import NaiveBayes # Maybe try other classification algorithms as well
from pyspark.ml import Pipeline

In [14]:
nb = NaiveBayes()

In [15]:
data_prep_pipe = Pipeline(stages=[
    ham_spam_to_numeric,
    tokenizer,
    stop_remover,
    count_vec,
    idf,
    assembler
])

In [16]:
pipe = data_prep_pipe.fit(data)

In [17]:
final_data = pipe.transform(data)

In [18]:
final_data.show(5)

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|class|                text|length|label|           tokenText|           stopToken|            countVec|               tfIDF|            features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|  0.0|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|
|  ham|Ok lar... Joking ...|    29|  0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,301,...|(13423,[0,24,301,...|(13424,[0,24,301,...|
| spam|Free entry in 2 a...|   155|  1.0|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[2,13,19,3...|
|  ham|U dun say so earl...|    49|  0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|(13423,[0,70,8

In [19]:
final_data = final_data.select('label', 'features')

In [20]:
final_data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
+-----+--------------------+
only showing top 5 rows



In [21]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [22]:
spam_detector = nb.fit(train_data)

24/01/22 22:32:33 WARN DAGScheduler: Broadcasting large task binary with size 1153.5 KiB
24/01/22 22:32:34 WARN DAGScheduler: Broadcasting large task binary with size 1133.0 KiB


In [23]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [24]:
test_results = spam_detector.transform(test_data)

In [25]:
test_results.show(5)

24/01/22 22:32:35 WARN DAGScheduler: Broadcasting large task binary with size 1358.5 KiB


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,7,8,1...|[-882.59543243869...|[1.0,1.8210356136...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1171.9991747457...|[1.0,3.9988727935...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1367.7548518705...|[1.0,1.6053676890...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-695.31133241105...|[1.0,9.3420492768...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-969.25048634492...|[1.0,7.1323746041...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



24/01/22 22:32:35 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/01/22 22:32:35 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


## Model Evaluation

In [26]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [27]:
evaluator = MulticlassClassificationEvaluator(metricName='accuracy')

In [28]:
accuracy = evaluator.evaluate(test_results)

24/01/22 22:32:36 WARN DAGScheduler: Broadcasting large task binary with size 1363.5 KiB


In [29]:
print(f"Model's accuracy : {accuracy * 100:.1f}%")

Model's accuracy : 91.6%
